<img src="instagram_logo.jpg" alt="Instagram Logo" width="300"/>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
import re

# User Posts

inside the `body` we are going to explore `items`

inside `items`:

* in `caption` we have all the text and the # - regex to get all the # only
* in `comment_count` we get the count of the comments to the post
* in `like_count` we get the count of the likes to the post
* in `location` extract place, city, facebookid

In [47]:
# from the list of profiles I colleted manually data
username = ['abcdosviajantes',
 'carimbacasado',
 'liviajando',
 'euvouporai',
 'dicadeturista',
 'viajareumprazer',
 'ateondeeupuderir',
 'vantravelpics',
 'euviajosozinha',
 'comigoporai',
  'travelbabbo',
  'Calsnape',
  'ournextflight',
  'takemyhearteverywhere',
  'danielkordan',
  'nobackhome',
  'chrisburkard',
  'melissa','funforlouis',
  'doyoutravel',
   'izkiz',
   'tuulavintage',
   'muradosmann',
   'stalman',
   'alenpalander',
   'benjaminortega',
   'jannid',
   'jonolsson1']
   #'lifeof_riley']
#len(username)

In [38]:
import time

url = "https://instagram47.p.rapidapi.com/user_posts/"

df = []

for user in username:
    querystring = {"username":str(user)}
    headers = {
        'x-rapidapi-key': "9054abd98dmshd8ac88b14b98c4fp18a496jsnd46bb5f9cd32",
        'x-rapidapi-host': "instagram47.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    time.sleep(5)
    raw_data = pd.json_normalize(response.json())
    df.append(raw_data)

In [39]:
data = pd.concat(df, axis=0).reset_index()
#data

In [40]:
items = pd.DataFrame.from_dict(data['body.items'])

#items.dropna().reset_index()
items

,body.items
0,"[{'taken_at': 1605716723, 'pk': 24452515666920..."
1,"[{'taken_at': 1605461792, 'pk': 24431130501389..."
2,"[{'taken_at': 1605632841, 'pk': 24445477598621..."
3,"[{'taken_at': 1605128932, 'pk': 24403208203280..."
4,"[{'taken_at': 1600042043, 'pk': 23976489007028..."
5,"[{'taken_at': 1605481235, 'pk': 24432761512866..."
6,"[{'taken_at': 1605646221, 'pk': 24446601478332..."
7,"[{'taken_at': 1604007353, 'pk': 24309123328490..."
8,"[{'taken_at': 1604333294, 'pk': 24336465206075..."
9,"[{'taken_at': 1605203166, 'pk': 24409435378134..."


In [41]:
df_data_list = []
r = 0
for row in range(len(df)):
    size = len(data['body.items'][row])
    r += size
    
    lst = items.loc[row].tolist()
    df_data = pd.DataFrame.from_dict(items.loc[row].tolist()).T
    df_data_list.append(df_data)
    
print('Total of photos from all profiles: ',r)
photos = pd.concat(df_data_list, axis=0).reset_index()
#photos

Total of photos from all profiles:  408


In [42]:
final_data = pd.DataFrame(dict(photos[0])).T
final_data.columns

Index(['taken_at', 'pk', 'id', 'device_timestamp', 'media_type', 'code',
       'client_cache_key', 'filter_type', 'location', 'lat', 'lng',
       'should_request_ads', 'user', 'can_viewer_reshare', 'caption_is_edited',
       'comment_likes_enabled', 'comment_threading_enabled',
       'has_more_comments', 'max_num_visible_preview_comments',
       'preview_comments', 'comments', 'can_view_more_preview_comments',
       'comment_count', 'inline_composer_display_condition', 'image_versions2',
       'original_width', 'original_height', 'like_count', 'has_liked',
       'top_likers', 'facepile_top_likers', 'photo_of_you',
       'can_see_insights_as_brand', 'caption', 'organic_tracking_token',
       'sharing_friction_info', 'is_in_profile_grid',
       'profile_grid_control_enabled', 'is_shop_the_look_eligible',
       'deleted_reason', 'integrity_review_decision', 'video_versions',
       'has_audio', 'video_duration', 'view_count', 'play_count',
       'product_type', 'clips_metadat

In [43]:
user = pd.DataFrame(dict(final_data['user'])).T
label = user[['pk','username']]
instagram_api = final_data[['like_count','comment_count']]
user_info = pd.concat([label, instagram_api], axis=1)
#user_info

In [44]:
location = pd.DataFrame(dict(final_data['location'])).T
local = location[['short_name','facebook_places_id','city','lng','lat']]

local_info = pd.concat([user_info, local], axis=1)
#local_info

In [49]:
caption = pd.DataFrame(dict(final_data['caption'])).T
text = caption['text']
api_info = pd.concat([local_info, text], axis=1)
api_info

,pk,username,like_count,comment_count,short_name,facebook_places_id,city,lng,lat,text
0,11704287485,abcdosviajantes,39,6,Escadaria Selarón,129293763779643,,-43.1795,-22.9154,Aqui vai um pouquinho da história da escadari...
1,11704287485,abcdosviajantes,66,27,Praia De Carro Quebrado,334376796650152,"Maceió, Brazil",-35.7074,-9.66124,Se você está curioso sobre a origem do nome ...
2,11704287485,abcdosviajantes,45,8,NaN,NaN,NaN,NaN,NaN,"A gastronomia no início de seu surgimento, er..."
3,11704287485,abcdosviajantes,65,34,Pedra Do Arpoador,141182515999238,,-43.1008,-22.8586,Você gosta de pôr do sol??? ☀️\n\nEntão aqu...
4,11704287485,abcdosviajantes,68,17,Maceió-Alagoas/ Nordeste Do Brasil,100437841374667,,-35.7243,-9.62537,"📍Dunas de Marapé, Alagoas.\n\nOntem perguntam..."
...,...,...,...,...,...,...,...,...,...,...
403,191681166,jonolsson1,208337,484,NaN,NaN,NaN,NaN,NaN,Cant think of a much better way to spend my 38...
404,191681166,jonolsson1,172751,304,Marbella,108331215864513,,-4.8855,36.509,Lord Hans The G is SOLD! 😳💔😭 \n\nAs much as I ...
405,191681166,jonolsson1,173081,407,Marbella,108331215864513,,-4.8855,36.509,100% Ultimate Driveway Goals! 💯❤️\n10.000.000€...
406,191681166,jonolsson1,237006,471,Marbella,108331215864513,,-4.8855,36.509,Yes dad! 👶🏼🙌🏼👶🏼 The driveway is now Hypercar t...


In [50]:
#api_info.to_excel('api_data.xlsx') # use that to generate only 10 profiles that I look the photos
api_info.to_excel('api_28_data.xlsx')